**Agent 2 for FE BE DB**

In [2]:
# OLLAMA Models Test -- Agent 2 (PRD → Architecture)

import requests

# Ollama API base
OLLAMA_URL = "http://localhost:11434/api/generate"

# Models to test for Agent 2
models = [
    "codellama:7b-instruct",
    "deepseek-coder:1.3b",
    "deepseek-coder:6.7b",
    "qwen2.5:7b-instruct",
    "mistral:7b-instruct",
    "phi3:3.8b",
    "starcoder2:3b",
    "starcoder2:7b"
]

# Instruction for Agent 2
# arch_instruction = """
# You are a Solutions Architect. Convert the following Product Requirement Document (PRD) into a structured Architecture Document with sections:
# - Components / Modules
# - Database Design
# - API Endpoints
# - Diagrams (Mermaid.js style if possible)
# - Tech Stack
# - Security & Access Control

# PRD:
# """

arch_instruction = """
You are a Solution Architect. Convert the following Product Requirement Document (PRD) into a **System Architecture Specification** with sections:

- Tech Stack (frontend, backend, database, third-party integrations)
- API Endpoints (CRUD + relevant features)
- Database Schema (tables, keys, fields)
- System Architecture Diagram (describe how components interact)

PRD:
"""


# Sample PRD text (output from Agent 1)
prd_text = """**Product Requirement Document (PRD)**

**Title:** Task Management Web Application

**Objectives:**
1. To streamline task management within the company.
2. To facilitate collaboration among team members by allowing task assignment and tracking.
3. To improve productivity by setting deadlines and sending automated reminders for pending tasks.
4. To provide managers with a dashboard to monitor task progress, view completed tasks, and analyze team performance.
5. To ensure secure access with role-based permissions (admin, manager, employee).
6. To make the application lightweight, fast, and responsive on both desktop and mobile devices.
7. To integrate the application with our existing email system for notifications.
8. To allow managers to download reports in CSV format.

**Key Features:**
1. Task creation and editing
2. Assignment of tasks to team members
3. Setting deadlines for tasks
4. Marking tasks as complete
5. Automated reminders for pending tasks
6. Manager dashboard for tracking task progress, viewing completed tasks, and analyzing team performance
7. Secure login
8. Role-based access (admin, manager, employee)
9. Responsive design for both desktop and mobile devices
10. Integration with the existing email system for notifications
11. Reporting functionality with CSV export option

**Acceptance Criteria:**
1. Users should be able to create, edit, assign, mark as complete, and delete tasks within the application.
2. Managers should be able to view all tasks assigned to their team members, including progress and completion status.
3. Automated reminders for pending tasks should be sent at the specified deadlines.
4. The manager dashboard should provide an overview of task progress, completed tasks, and team performance.
5. The application should only allow authorized users (admin or manager) to access certain features based on their roles.
6. The application should function correctly across multiple devices and screen sizes.
7. Integration with the existing email system for notifications should be seamless and reliable.
8. Reports should be downloadable in CSV format from the manager dashboard.

**Integrations:**
1. Existing Email System (SMTP)

**Reporting:**
1. Task progress reports
2. Completed task reports
3. Team performance reports in CSV format
"""

prompt = arch_instruction + prd_text

def query_model(model, prompt):
    """Send prompt to Ollama model and return response"""
    try:
        response = requests.post(OLLAMA_URL, json={
            "model": model,
            "prompt": prompt,
            "stream": False
        })
        response.raise_for_status()
        return response.json().get("response", "").strip()
    except Exception as e:
        return f"❌ Error with {model}: {str(e)}"

def score_architecture(output):
    """
    Score the Architecture output for:
    1. Section presence
    2. Word length
    3. Coverage of key architecture elements
    """
    # 1️⃣ Section check
    sections = ["Components", "Database", "API", "Diagrams", "Tech Stack", "Security"]
    section_score = sum(1 for sec in sections if sec.lower() in output.lower())

    # 2️⃣ Length check
    length = len(output.split())
    length_score = 1 if 200 <= length <= 500 else 0

    # 3️⃣ Accuracy check (keywords from PRD)
    key_points = [
        "task management module", "dashboard", "user roles", "email integration",
        "database", "API", "security", "responsive"
    ]
    matched = sum(1 for point in key_points if point.lower() in output.lower())
    accuracy_score = matched / len(key_points)  # normalized 0–1

    # 4️⃣ Total score
    total_score = section_score + length_score + accuracy_score * 4  # section+length max=7, accuracy max=4
    return total_score, section_score, length, accuracy_score

# Run models and store results
results = []

for model in models:
    print("="*60)
    print(f"🚀 Model: {model}")
    print("-"*60)
    output = query_model(model, prompt)
    print(output)
    total, sec_score, length, acc_score = score_architecture(output)
    print(f"✅ Total Score: {total:.2f}/11 | Sections: {sec_score}/6 | Length: {length} words | Accuracy: {acc_score*100:.0f}%\n")
    results.append((model, total))

# Rank models
results.sort(key=lambda x: x[1], reverse=True)
best_model = results[0][0]

print("="*60)
print(f"🏆 Best model for Agent 2: {best_model}")


🚀 Model: codellama:7b-instruct
------------------------------------------------------------
Title: Task Management Web Application

Objectives:
1. To streamline task management within the company.
2. To facilitate collaboration among team members by allowing task assignment and tracking.
3. To improve productivity by setting deadlines and sending automated reminders for pending tasks.
4. To provide managers with a dashboard to monitor task progress, view completed tasks, and analyze team performance.
5. To ensure secure access with role-based permissions (admin, manager, employee).
6. To make the application lightweight, fast, and responsive on both desktop and mobile devices.
7. To integrate the application with our existing email system for notifications.
8. To allow managers to download reports in CSV format.

Tech Stack:
Frontend: HTML, CSS, JavaScript (React), Typescript
Backend: NodeJS, Express, MongoDB
Database: MongoDB
Third-Party Integrations: SMTP (Existing Email System)
API 

**Agent 2 - only for frontend updated**

In [3]:
import requests
import re

# Ollama API base
OLLAMA_URL = "http://localhost:11434/api/generate"

# Models to test (small, code-focused, quantized for 16GB RAM)
models = [
    "codellama:7b-instruct",
    "deepseek-coder:6.7b",
    "qwen2.5:7b-instruct",
    "mistral:7b-instruct",
    "phi3:3.8b",
    "qwen2.5-coder:7b" ,
    "starcoder2:7b"
]

# Updated instruction for frontend-only architecture
arch_instruction = """
You are a Solutions Architect. Convert the following Product Requirement Document (PRD) into a **System Architecture Specification** for a frontend-only React application. Include these sections:
- Tech Stack (frontend framework, styling, build tools, libraries)
- Components (React components for UI)
- Data Storage (client-side storage mechanism)
- UI Features (responsive design, notifications, reports, access control)

Use React with Redux for state management, Tailwind CSS for styling, Vite for builds, and localStorage for data persistence. Simulate backend features (e.g., notifications, authentication) client-side using browser alerts, modals, or localStorage. Output a structured markdown document. PRD:
"""

# Sample PRD text (unchanged, but interpreted as frontend-only)
prd_text = """**Product Requirement Document (PRD)**

**Title:** Task Management Web Application

**Objectives:**
1. To streamline task management within the company.
2. To facilitate collaboration among team members by allowing task assignment and tracking.
3. To improve productivity by setting deadlines and sending automated reminders for pending tasks.
4. To provide managers with a dashboard to monitor task progress, view completed tasks, and analyze team performance.
5. To ensure secure access with role-based permissions (admin, manager, employee).
6. To make the application lightweight, fast, and responsive on both desktop and mobile devices.
7. To integrate the application with our existing email system for notifications.
8. To allow managers to download reports in CSV format.

**Key Features:**
1. Task creation and editing
2. Assignment of tasks to team members
3. Setting deadlines for tasks
4. Marking tasks as complete
5. Automated reminders for pending tasks
6. Manager dashboard for tracking task progress, viewing completed tasks, and analyzing team performance
7. Secure login
8. Role-based access (admin, manager, employee)
9. Responsive design for both desktop and mobile devices
10. Integration with the existing email system for notifications
11. Reporting functionality with CSV export option

**Acceptance Criteria:**
1. Users should be able to create, edit, assign, mark as complete, and delete tasks within the application.
2. Managers should be able to view all tasks assigned to their team members, including progress and completion status.
3. Automated reminders for pending tasks should be sent at the specified deadlines.
4. The manager dashboard should provide an overview of task progress, completed tasks, and team performance.
5. The application should only allow authorized users (admin or manager) to access certain features based on their roles.
6. The application should function correctly across multiple devices and screen sizes.
7. Integration with the existing email system for notifications should be seamless and reliable.
8. Reports should be downloadable in CSV format from the manager dashboard.

**Integrations:**
1. Existing Email System (SMTP)

**Reporting:**
1. Task progress reports
2. Completed task reports
3. Team performance reports in CSV format
"""

prompt = arch_instruction + prd_text

def query_model(model, prompt):
    """Send prompt to Ollama model and return response"""
    try:
        response = requests.post(OLLAMA_URL, json={
            "model": model,
            "prompt": prompt,
            "stream": False,
            "options": {"num_ctx": 4096}  # Increase context window
        })
        response.raise_for_status()
        return response.json().get("response", "").strip()
    except Exception as e:
        return f"❌ Error with {model}: {str(e)}"

def score_architecture(output):
    """
    Score the architecture output for:
    1. Section presence (Tech Stack, Components, Data Storage, UI Features)
    2. Word length
    3. Coverage of key architecture elements
    """
    # 1️⃣ Section check
    sections = ["Tech Stack", "Components", "Data Storage", "UI Features"]
    section_score = sum(1 for sec in sections if sec.lower() in output.lower())

    # 2️⃣ Length check
    length = len(output.split())
    length_score = 1 if 150 <= length <= 400 else 0  # Adjusted for frontend-only

    # 3️⃣ Accuracy check (frontend-specific keywords from PRD)
    key_points = [
        "react", "redux", "tailwind css", "vite", "localstorage", "papaparse",
        "taskform", "tasklist", "managerdashboard", "responsive design",
        "role-based access", "csv report", "notifications", "error handling"
    ]
    matched = sum(1 for point in key_points if point.lower() in output.lower())
    accuracy_score = matched / len(key_points) if key_points else 0  # normalized 0–1

    # 4️⃣ Total score
    total_score = section_score + length_score + accuracy_score * 4  # max=4+1+4=9
    return total_score, section_score, length, accuracy_score

# Run models and store results
results = []

for model in models:
    print("="*60)
    print(f"🚀 Model: {model}")
    print("-"*60)
    output = query_model(model, prompt)
    print(output)
    total, sec_score, length, acc_score = score_architecture(output)
    print(f"✅ Total Score: {total:.2f}/9 | Sections: {sec_score}/4 | Length: {length} words | Accuracy: {acc_score*100:.0f}%\n")
    results.append((model, total))

# Rank models
results.sort(key=lambda x: x[1], reverse=True)
best_model = results[0][0]

print("="*60)
print(f"🏆 Best model for Agent 2: {best_model}")

🚀 Model: codellama:7b-instruct
------------------------------------------------------------
System Architecture Specification for a Frontend-only React Application

Introduction
------------

The Task Management Web Application is designed to streamline task management within the company, facilitate collaboration among team members by allowing task assignment and tracking, improve productivity by setting deadlines and sending automated reminders for pending tasks, provide managers with a dashboard to monitor task progress, view completed tasks, and analyze team performance, ensure secure access with role-based permissions (admin, manager, employee), make the application lightweight, fast, and responsive on both desktop and mobile devices, integrate the application with our existing email system for notifications, allow managers to download reports in CSV format.

Tech Stack
-----------

* Frontend framework: React with Redux
* Styling: Tailwind CSS
* Build tools: Vite
* Libraries: loca

**BRD 2**

In [ ]:
import requests
import re

# Ollama API base
OLLAMA_URL = "http://localhost:11434/api/generate"

# Models to test (small, code-focused, quantized for 16GB RAM)
models = [
    "codellama:7b-instruct",
    "deepseek-coder:6.7b",
    "qwen2.5:7b-instruct",
    "mistral:7b-instruct",
    "phi3:3.8b",
    "qwen2.5-coder:7b" 
]

# Updated instruction for frontend-only architecture
arch_instruction = """
You are a Solutions Architect. Convert the following Product Requirement Document (PRD) into a **System Architecture Specification** for a frontend-only React application. Include these sections:
- Tech Stack (frontend framework, styling, build tools, libraries)
- Components (React components for UI)
- Data Storage (client-side storage mechanism)
- UI Features (responsive design, notifications, reports, access control)

Use React with Redux for state management, Tailwind CSS for styling, Vite for builds, and localStorage for data persistence. Simulate backend features (e.g., notifications, authentication) client-side using browser alerts, modals, or localStorage. Output a structured markdown document. PRD:
"""

# Sample PRD text (unchanged, but interpreted as frontend-only)
prd_text = """**Product Requirement Document (PRD)**

Here is the converted Product Requirement Document (PRD) in a structured format:

**Objectives**

The objective of this project is to build a modern, responsive, and visually appealing personal portfolio website that showcases an individual's projects and skills. The website should provide a seamless user experience with smooth navigation and animations.

**UI Components**

* Home: A hero section with a background image/video, heading, and brief summary
* About: A bio section with a profile picture, heading, and short description
* Projects:
	+ Project cards with images, titles, short descriptions, and buttons to view more details or visit external links (e.g., GitHub, Live Demo)
	+ Pagination for displaying multiple projects
* Skills: A list of skills with icons and brief descriptions
* Contact: A simple form with fields for name, email, and message

**State Management Requirements**

* Redux setup with the following slices:
	+ ProjectsSlice: manages project data (title, description, image, etc.)
	+ ContactFormSlice: manages contact form state (name, email, message)
* Actions:
	+ FetchProjectsAction: fetches project data from storage (localStorage) or API (if needed)
	+ UpdateContactFormDataAction: updates the contact form state
* Reducers:
	+ ProjectsReducer: handles updating project data
	+ ContactFormReducer: handles updating the contact form state

**Data Persistence**

* Store project data in localStorage for fast access and retrieval
* Use JSON.stringify() and JSON.parse() to serialize and deserialize project data, respectively

**User Roles & Access Control**

* No user roles or access control required as this is a single-user portfolio website with no login functionality.

**Acceptance Criteria**

1. The website should have a smooth navigation experience using single-page scrolling.
2. The Projects section should display project cards with images, titles, short descriptions, and buttons to view more details or visit external links.
3. The Contact section should include a simple form with fields for name, email, and message.
4. The website should be responsive on desktop, tablet, and mobile devices.
5. Animations and transitions should be used for smooth user experience.
6. The website should load quickly (< 2 seconds) and have a lightweight footprint (< 500 KB).
7. The design should be visually appealing with a modern color scheme.
"""

prompt = arch_instruction + prd_text

def query_model(model, prompt):
    """Send prompt to Ollama model and return response"""
    try:
        response = requests.post(OLLAMA_URL, json={
            "model": model,
            "prompt": prompt,
            "stream": False,
            "options": {"num_ctx": 4096}  # Increase context window
        })
        response.raise_for_status()
        return response.json().get("response", "").strip()
    except Exception as e:
        return f"❌ Error with {model}: {str(e)}"

def score_architecture(output):
    """
    Score the architecture output for:
    1. Section presence (Tech Stack, Components, Data Storage, UI Features)
    2. Word length
    3. Coverage of key architecture elements
    """
    # 1️⃣ Section check
    sections = ["Tech Stack", "Components", "Data Storage", "UI Features"]
    section_score = sum(1 for sec in sections if sec.lower() in output.lower())

    # 2️⃣ Length check
    length = len(output.split())
    length_score = 1 if 150 <= length <= 400 else 0  # Adjusted for frontend-only

    # 3️⃣ Accuracy check (frontend-specific keywords from PRD)
    key_points = [
        "react", "redux", "tailwind css", "vite", "localstorage", "papaparse",
        "taskform", "tasklist", "managerdashboard", "responsive design",
        "role-based access", "csv report", "notifications", "error handling"
    ]
    matched = sum(1 for point in key_points if point.lower() in output.lower())
    accuracy_score = matched / len(key_points) if key_points else 0  # normalized 0–1

    # 4️⃣ Total score
    total_score = section_score + length_score + accuracy_score * 4  # max=4+1+4=9
    return total_score, section_score, length, accuracy_score

# Run models and store results
results = []

for model in models:
    print("="*60)
    print(f"🚀 Model: {model}")
    print("-"*60)
    output = query_model(model, prompt)
    print(output)
    total, sec_score, length, acc_score = score_architecture(output)
    print(f"✅ Total Score: {total:.2f}/9 | Sections: {sec_score}/4 | Length: {length} words | Accuracy: {acc_score*100:.0f}%\n")
    results.append((model, total))

# Rank models
results.sort(key=lambda x: x[1], reverse=True)
best_model = results[0][0]

print("="*60)
print(f"🏆 Best model for Agent 2: {best_model}")

🚀 Model: codellama:7b-instruct
------------------------------------------------------------
**Product Requirement Document (PRD) for Frontend-Only React Application**

Objectives
-----------
The objective of this project is to build a modern, responsive, and visually appealing personal portfolio website that showcases an individual's projects and skills. The website should provide a seamless user experience with smooth navigation and animations.

Tech Stack
-------------
* Frontend Framework: React
* State Management: Redux
* Styling: Tailwind CSS
* Build Tools: Vite
* Libraries: localStorage for data persistence

Components
-----------

1. Home (Hero Section)
2. About (Bio Section)
3. Projects
	* Project Cards with Images, Titles, Short Descriptions, and Buttons to View More Details or Visit External Links
	* Pagination for Displaying Multiple Projects
4. Skills
	* List of Skills with Icons and Brief Descriptions
5. Contact
	* Simple Form with Fields for Name, Email, and Message

Stat

**BRD 2 Updated**

In [ ]:
import requests
import re

# Ollama API base
OLLAMA_URL = "http://localhost:11434/api/generate"

# Models to test (small, code-focused, quantized for 16GB RAM)
models = [
    "codellama:7b-instruct",
    "deepseek-coder:6.7b",
    "mistral:7b-instruct",
    "phi3:3.8b",
    "qwen2.5-coder:7b" 
]



arch_instruction = """
You are a Solutions Architect. Convert the following Product Requirement Document (PRD)
into a developer-focused **Frontend Architecture Specification** that explicitly maps features to file paths.

Rules:
- The frontend must always be implemented in **React**, regardless of what the PRD says.
- Styling must use **Tailwind CSS**.
- Build tool must be **Vite**.
- State management should default to **Redux Toolkit** (or Context API if very simple).
- Client-side storage should be **localStorage/sessionStorage** if data persistence is needed.
- All screens and components must be explicitly mapped to file paths (e.g., src/screens/Home.jsx, src/components/Navbar.jsx).
- Mandatory base files must always be listed under "Mandatory Files".

Your architecture document must contain ONLY the following sections:
1. Tech Stack (React, Tailwind, Vite, state management, other required libs)
2. Mandatory Files (always present: src/App.jsx, src/main.jsx, src/index.css, tailwind.config.js, vite.config.js, package.json)
3. Screens (each screen listed in PRD → mapped to src/screens/*.jsx)
4. Components (each reusable component → mapped to src/components/*.jsx)
5. State Management (Redux store and slices if needed, with file paths)
6. Data Storage (localStorage/sessionStorage if applicable, otherwise None)
7. UI Features (responsive design, animations, forms, role-based access if mentioned)

Output must be a clean markdown document with file mappings.
"""


# Sample PRD text (unchanged, but interpreted as frontend-only)
prd_text = """Here is the converted PRD: 

Objectives 

• Build a personal portfolio website with 5 main sections: Home, About, Projects, Skills, and Contact  

• Implement single-page scrolling navigation  

• Create a modern, responsive, and lightweight design 

UI Screens & Components 

• Home screen  

• About screen  

• Projects screen (with project cards)  

• Skills screen  

• Contact form  

• Project card (with image, title, description, and buttons) 

State Management 

• Navigation state: current section • Project details state: expanded/collapsed 

Data Persistence 

None (frontend-only project) 

User Roles 

None 

Acceptance Criteria 

• Navigation links at the top of the page allow smooth scrolling between sections  

• Projects screen displays project cards with images, titles, descriptions, and buttons  

• Contact form includes fields for name, email, and message  

• Design is modern, responsive, and lightweight  

• Animations and transitions are used for a smooth user experience 
"""

prompt = arch_instruction + prd_text

def query_model(model, prompt):
    """Send prompt to Ollama model and return response"""
    try:
        response = requests.post(OLLAMA_URL, json={
            "model": model,
            "prompt": prompt,
            "stream": False,
            "options": {"num_ctx": 4096}  # Increase context window
        })
        response.raise_for_status()
        return response.json().get("response", "").strip()
    except Exception as e:
        return f"❌ Error with {model}: {str(e)}"

import re
import nltk
from nltk.corpus import stopwords
from collections import Counter

# Run once in setup
nltk.download("punkt")
nltk.download("stopwords")

def extract_key_features(prd_text, top_n=12):
    """
    Extract candidate key features (nouns/important words) from PRD text.
    Uses simple frequency analysis after removing stopwords.
    """
    words = re.findall(r"\b\w+\b", prd_text.lower())
    stop_words = set(stopwords.words("english"))
    words = [w for w in words if w not in stop_words and len(w) > 2]
    
    common = Counter(words).most_common(top_n)
    features = [word for word, _ in common]
    return features

def score_architecture(output, prd_text):
    """
    Score the Frontend Architecture output for:
    1. Section presence
    2. Word length
    3. Accuracy: coverage of PRD features
    4. Penalty for violating React-only rule
    """
    # 1️⃣ Required sections for frontend arch
    sections = [
        "Tech Stack",
        "Components",
        "State Management",
        "Data Storage",
        "UI Features"
    ]
    section_score = sum(1 for sec in sections if sec.lower() in output.lower())

    # 2️⃣ Length check
    length = len(output.split())
    length_score = 1 if 150 <= length <= 400 else 0

    # 3️⃣ Dynamic Accuracy check
    key_features = extract_key_features(prd_text)
    matched_features = sum(1 for feat in key_features if feat in output.lower())
    accuracy_score = matched_features / len(key_features) if key_features else 0

    # 4️⃣ React-only penalty (negative points if other frameworks show up)
    forbidden = ["angular", "vue", "next.js", "svelte"]
    penalty = -1 if any(f in output.lower() for f in forbidden) else 0

    # 5️⃣ Final total score
    total_score = section_score + length_score + accuracy_score * 3 + penalty
    return total_score, section_score, length, accuracy_score, key_features


# Run models and store results
results = []

for model in models:
    print("="*60)
    print(f"🚀 Model: {model}")
    print("-"*60)
    output = query_model(model, prompt)
    print(output)
    total, sec_score, length, acc_score, key_features = score_architecture(output, prd_text)
    print(f"✅ Total Score: {total:.2f}/9 | Sections: {sec_score}/5 | Length: {length} words | Accuracy: {acc_score*100:.0f}%")
    print(f"🔑 PRD Key Features: {key_features}\n")
    results.append((model, total))

# Rank models
results.sort(key=lambda x: x[1], reverse=True)
best_model = results[0][0]

print("="*60)
print(f"🏆 Best model for Agent 2: {best_model}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CDS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CDS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


🚀 Model: codellama:7b-instruct
------------------------------------------------------------
Tech Stack:
React
Tailwind CSS
Vite
Redux Toolkit (state management)
localStorage/sessionStorage (for data persistence)

Mandatory Files:
src/App.jsx
src/main.jsx
src/index.css
tailwind.config.js
vite.config.js
package.json

Screens:
src/screens/Home.jsx
src/screens/About.jsx
src/screens/Projects.jsx
src/screens/Skills.jsx
src/screens/Contact.jsx

Components:
src/components/Navbar.jsx (navigation bar)
src/components/ProjectCard.jsx (project card component)
src/components/ContactForm.jsx (contact form component)

State Management:
Redux store and slices for navigation state and project details state

Data Storage:
localStorage/sessionStorage for data persistence

UI Features:
Responsive design
Animations and transitions for smooth user experience
✅ Total Score: 7.00/9 | Sections: 5/5 | Length: 70 words | Accuracy: 67%
🔑 PRD Key Features: ['screen', 'project', 'projects', 'contact', 'navigation', 

**Example flow**

In [ ]:
# # Full Pipeline: BRD → PRD → Architecture → Code
# import requests
# from crewai import Agent, Task, Crew

# # Ollama API base
# OLLAMA_URL = "http://localhost:11434/api/generate"

# # Models (quantized for 16GB RAM)
# models = {
#     "agent1": "mistral:7b-instruct-q4_0",  # ~4GB
#     "agent2": "phi3:3.8b",                 # ~2.5GB
#     "agent3": "deepseek-coder:1.3b"        # ~1GB
# }

# def query_model(model, prompt):
#     """Send prompt to Ollama model and return response"""
#     try:
#         response = requests.post(OLLAMA_URL, json={
#             "model": model,
#             "prompt": prompt,
#             "stream": False
#         })
#         response.raise_for_status()
#         return response.json().get("response", "").strip()
#     except Exception as e:
#         return f"❌ Error with {model}: {str(e)}"

# # Instructions (reusing Agent 1 and 2, adding Agent 3)
# prd_instruction = """
# You are a Product Manager. Convert the following Business Requirement Document (BRD) into a structured Product Requirement Document (PRD) with sections:
# - Objectives
# - Key Features
# - Acceptance Criteria
# - Integrations
# - Reporting
# BRD:
# """

# arch_instruction = """
# You are a Solution Architect. Convert the following Product Requirement Document (PRD) into a System Architecture Specification with sections:
# - Tech Stack (frontend, backend, database, third-party integrations)
# - API Endpoints (CRUD + relevant features)
# - Database Schema (tables, keys, fields)
# - System Architecture Diagram (describe how components interact)
# PRD:
# """

# code_instruction = """
# You are a Full-Stack Developer. Generate complete code for a Next.js 14 frontend, Express 4.x backend, and PostgreSQL 15 database based on the provided System Architecture Specification. Output as a JSON object with keys: `frontend`, `backend`, `database`. Each key should contain files as sub-keys (e.g., `frontend: { "pages/index.js": "...", "components/TaskForm.js": "..." }`).
# Architecture:
# """

# # Sample BRD (same as before)
# brd_text = "We want to build a task management web application for our employees. Users should be able to create tasks, assign them to team members, set deadlines, and mark tasks as complete. The app should send automated reminders for pending tasks. Managers need a dashboard to track task progress, view completed tasks, and analyze team performance. The application must have secure login, responsive design for both desktop and mobile, and role-based access (admin, manager, employee). Reports should be downloadable in CSV format. The system should be lightweight, fast, and integrate with our existing email system for notifications."

# # CrewAI Setup
# agent1 = Agent(
#     role='Product Manager',
#     goal='Convert BRD to structured PRD',
#     backstory='Expert at formalizing business requirements.',
#     llm=models["agent1"],
#     function=query_model
# )
# agent2 = Agent(
#     role='Solution Architect',
#     goal='Create architecture spec from PRD',
#     backstory='Skilled at designing tech stacks and APIs.',
#     llm=models["agent2"],
#     function=query_model
# )
# agent3 = Agent(
#     role='Full-Stack Developer',
#     goal='Generate full-stack code',
#     backstory='Expert at coding Next.js, Express, and PostgreSQL.',
#     llm=models["agent3"],
#     function=query_model
# )

# task1 = Task(
#     description=prd_instruction + brd_text,
#     agent=agent1
# )
# task2 = Task(
#     description=arch_instruction + "{task1_output}",
#     agent=agent2
# )
# task3 = Task(
#     description=code_instruction + "{task2_output}",
#     agent=agent3
# )

# # Run the crew
# crew = Crew(
#     agents=[agent1, agent2, agent3],
#     tasks=[task1, task2, task3],
#     verbose=True
# )
# result = crew.kickoff()

# print("="*60)
# print(f"PRD:\n{result.task1_output}")
# print(f"Architecture:\n{result.task2_output}")
# print(f"Code:\n{result.task3_output}")

In [ ]:
# import requests
# from crewai import Agent, Task, Crew

# # Ollama API base
# OLLAMA_URL = "http://localhost:11434/api/generate"

# # Models (quantized for 16GB RAM)
# models = {
#     "agent1": "mistral:7b-instruct-q4_0",  # ~4GB
#     "agent2": "phi3:3.8b",                 # ~2.5GB
#     "agent3": "deepseek-coder:1.3b"        # ~1GB
# }

# def query_model(model, prompt):
#     """Send prompt to Ollama model and return response"""
#     try:
#         response = requests.post(OLLAMA_URL, json={
#             "model": model,
#             "prompt": prompt,
#             "stream": False
#         })
#         response.raise_for_status()
#         return response.json().get("response", "").strip()
#     except Exception as e:
#         return f"❌ Error with {model}: {str(e)}"

# # Instructions
# prd_instruction = """
# You are a Product Manager. Convert the following Business Requirement Document (BRD) into a structured Product Requirement Document (PRD) with sections:
# - Objectives
# - Key Features
# - Acceptance Criteria
# - Integrations
# - Reporting
# BRD:
# """

# arch_instruction = """
# You are a Solution Architect. Convert the following Product Requirement Document (PRD) into a System Architecture Specification with sections:
# - Tech Stack (frontend, backend, database, third-party integrations)
# - API Endpoints (CRUD + relevant features)
# - Database Schema (tables, keys, fields)
# - System Architecture Diagram (describe how components interact)
# PRD:
# """

# code_instruction = """
# You are a Full-Stack Developer. Generate complete code for a Next.js 14 frontend, Express 4.x backend, and PostgreSQL 15 database based on the provided System Architecture Specification. Output as a JSON object with keys: `frontend`, `backend`, `database`. Each key should contain files as sub-keys (e.g., `frontend: { "pages/index.js": "...", "components/TaskForm.js": "..." }`). Include error handling, JWT authentication, and Nodemailer for email integration. Ensure code is production-ready and follows best practices.
# Architecture:
# """

# # Sample BRD (from your Agent 1 script)
# brd_text = """We want to build a task management web application for our employees. Users should be able to create tasks, assign them to team members, set deadlines, and mark tasks as complete. The app should send automated reminders for pending tasks. Managers need a dashboard to track task progress, view completed tasks, and analyze team performance. The application must have secure login, responsive design for both desktop and mobile, and role-based access (admin, manager, employee). Reports should be downloadable in CSV format. The system should be lightweight, fast, and integrate with our existing email system for notifications."""

# # Sample PRD (from your Agent 2 script)
# prd_text = """**Product Requirement Document (PRD)**

# **Title:** Task Management Web Application

# **Objectives:**
# 1. To streamline task management within the company.
# 2. To facilitate collaboration among team members by allowing task assignment and tracking.
# 3. To improve productivity by setting deadlines and sending automated reminders for pending tasks.
# 4. To provide managers with a dashboard to monitor task progress, view completed tasks, and analyze team performance.
# 5. To ensure secure access with role-based permissions (admin, manager, employee).
# 6. To make the application lightweight, fast, and responsive on both desktop and mobile devices.
# 7. To integrate the application with our existing email system for notifications.
# 8. To allow managers to download reports in CSV format.

# **Key Features:**
# 1. Task creation and editing
# 2. Assignment of tasks to team members
# 3. Setting deadlines for tasks
# 4. Marking tasks as complete
# 5. Automated reminders for pending tasks
# 6. Manager dashboard for tracking task progress, viewing completed tasks, and analyzing team performance
# 7. Secure login
# 8. Role-based access (admin, manager, employee)
# 9. Responsive design for both desktop and mobile devices
# 10. Integration with the existing email system for notifications
# 11. Reporting functionality with CSV export option

# **Acceptance Criteria:**
# 1. Users should be able to create, edit, assign, mark as complete, and delete tasks within the application.
# 2. Managers should be able to view all tasks assigned to their team members, including progress and completion status.
# 3. Automated reminders for pending tasks should be sent at the specified deadlines.
# 4. The manager dashboard should provide an overview of task progress, completed tasks, and team performance.
# 5. The application should only allow authorized users (admin or manager) to access certain features based on their roles.
# 6. The application should function correctly across multiple devices and screen sizes.
# 7. Integration with the existing email system for notifications should be seamless and reliable.
# 8. Reports should be downloadable in CSV format from the manager dashboard.

# **Integrations:**
# 1. Existing Email System (SMTP)

# **Reporting:**
# 1. Task progress reports
# 2. Completed task reports
# 3. Team performance reports in CSV format
# """

# # Sample Architecture (assumed from Agent 2)
# arch_text = """**System Architecture Specification**

# **Tech Stack**:
# - Frontend: Next.js 14 (React), Tailwind CSS for responsive UI.
# - Backend: Express 4.x (Node.js).
# - Database: PostgreSQL 15.
# - Third-Party Integrations: SMTP for email notifications (e.g., Nodemailer).

# **API Endpoints**:
# - POST /api/tasks: Create a new task (title, deadline, assigned_to, status).
# - GET /api/tasks: List all tasks for a user or manager.
# - PUT /api/tasks/:id: Update a task (e.g., mark as complete).
# - DELETE /api/tasks/:id: Delete a task.
# - GET /api/reports: Generate CSV report of task progress or team performance.
# - POST /api/auth/login: Authenticate user (returns JWT token).
# - GET /api/dashboard: Fetch dashboard data (task progress, completed tasks).

# **Database Schema**:
# - Table: users
#   - id (SERIAL PRIMARY KEY)
#   - username (TEXT, UNIQUE)
#   - password (TEXT, hashed)
#   - role (TEXT, ENUM: admin, manager, employee)
# - Table: tasks
#   - id (SERIAL PRIMARY KEY)
#   - title (TEXT)
#   - description (TEXT)
#   - deadline (DATE)
#   - assigned_to (INTEGER, FOREIGN KEY to users.id)
#   - status (TEXT, ENUM: pending, completed)
#   - created_at (TIMESTAMP, DEFAULT CURRENT_TIMESTAMP)
# - Table: reports
#   - id (SERIAL PRIMARY KEY)
#   - type (TEXT, ENUM: progress, completed, performance)
#   - data (JSONB)
#   - created_at (TIMESTAMP, DEFAULT CURRENT_TIMESTAMP)

# **System Architecture Diagram**:
# - Frontend (Next.js + Tailwind) sends HTTP requests to Backend (Express).
# - Backend handles authentication (JWT), processes API requests, and queries PostgreSQL.
# - PostgreSQL stores users, tasks, and reports.
# - Backend uses Nodemailer to send email reminders via SMTP.
# - Manager dashboard fetches data via GET /api/dashboard and displays in responsive UI.
# - Reports are generated as CSV files via GET /api/reports.
# """

# # CrewAI Setup
# agent1 = Agent(
#     role='Product Manager',
#     goal='Convert BRD to structured PRD',
#     backstory='Expert at formalizing business requirements.',
#     llm=models["agent1"],
#     function=query_model
# )
# agent2 = Agent(
#     role='Solution Architect',
#     goal='Create architecture spec from PRD',
#     backstory='Skilled at designing tech stacks and APIs.',
#     llm=models["agent2"],
#     function=query_model
# )
# agent3 = Agent(
#     role='Full-Stack Developer',
#     goal='Generate full-stack code',
#     backstory='Expert at coding Next.js, Express, and PostgreSQL.',
#     llm=models["agent3"],
#     function=query_model
# )

# task1 = Task(
#     description=prd_instruction + brd_text,
#     agent=agent1
# )
# task2 = Task(
#     description=arch_instruction + prd_text,
#     agent=agent2
# )
# task3 = Task(
#     description=code_instruction + arch_text,
#     agent=agent3
# )

# # Run the crew
# crew = Crew(
#     agents=[agent1, agent2, agent3],
#     tasks=[task1, task2, task3],
#     verbose=True
# )
# result = crew.kickoff()

# print("="*60)
# print(f"PRD:\n{result.task1_output}")
# print(f"Architecture:\n{result.task2_output}")
# print(f"Code:\n{result.task3_output}")

In [ ]:
# import requests
# import json

# # Ollama API base
# OLLAMA_URL = "http://localhost:11434/api/generate"

# # Models to test (small, code-focused, quantized for 16GB RAM)
# models = [
#     "deepseek-coder:1.3b",
#     "deepseek-coder:6.7b",        # ~1GB, code-focused
#     "starcoder2:3b",      
#     "starcoder2:7b",       # ~2GB, code-focused
#     "codellama:7b-instruct",     # ~4GB, code-focused
#     "qwen2.5:0.5b-instruct",
#     "qwen2.5-coder:7b"     # ~300MB, general-purpose (backup)
# ]

# # Instruction for Agent 3
# code_instruction = """
# You are a Full-Stack Developer. Generate complete code for the frontend, backend, and database based on the provided System Architecture Specification. Use the technologies specified in the architecture's Tech Stack section (e.g., frontend framework, backend framework, database type). Output as a valid JSON object with keys: `frontend`, `backend`, `database`. Each key should contain files as sub-keys (e.g., `frontend: { "index.js": "...", "components/TaskForm.js": "..." }`). Include:
# - Frontend: Responsive UI, authentication, task creation/listing, and manager dashboard as per the architecture.
# - Backend: Authentication, API endpoints, email notifications, and report generation as specified.
# - Database: Schema creation scripts matching the architecture's Database Schema.
# Ensure code is production-ready, includes error handling, and follows best practices. Output must be valid JSON.
# Architecture:
# """

# # Sample architecture text (from your input)
# arch_text = """**System Architecture Specification for "Task Management Web Application" based on the given Product Requirement Document (PRD).**

# ### Tech Stack ###

# - **Frontend:** React with Redux Framework, CSS Flexbox and Grid. It will be optimized using code splitting and lazy loading to ensure lightweight performance across devices/screens sizes. 

# - **Backend:** Node.js Express Server running on a secure HTTPS environment (TLS encryption). This server will expose RESTful API endpoints for the CRUD operations of tasks, team members' assignments and role access controls. The backend is using PostgreSQL as its database with Sequelize ORM tool to handle SQL queries effectively.

# - **Database:** PostgreSQL Database designed in a way that optimizes read/write performance by creating separate indices for frequently queried data (task status, deadlines etc.) and implementing foreign key constraints between the tables Tasks, Team_members, Roles etc. 

# - **Third Party Integrations:** The application will integrate with an existing SMTP email system to send notifications through emails using Nodemailer - a popular Email Library for node that provides easy integration of SMTP servers and APIs within our Express Backend server itself without any external API calls, ensuring seamless notification delivery.

# ### System Architecture Diagram ### 
# The architecture diagram would show the client-side React App interacting with various backend endpoints via HTTPS (Secure RESTful services). The main components of this system are as follows: Task Management Service on our Node.js server, a PostgreSQL Database serving data to both Services and also providing an API endpoint for generating CSV reports which is consumed by the manager dashboard component built with React Native's capabilities ensuring mobile compatibility due its lightweight nature. The User Roles (Admin/Manager) are managed at Service level where each user login attempt triggers Role-based access control checks before granting or denying permissions to specific features and data based on their role, this is done via middleware in Express backend that intercepts all incoming requests for the various endpoints.

# ### API Endpoints ### 
# 1. `POST /api/tasks` - Create a task with relevant details such as title, description etc., by default it will assign to unassigned team members and set deadline based on input parameters or defaults (if not provided).
# 2. `GET /api/tasks/{id}` - Read operation for getting the complete list of tasks assigned to specific user(s) along with their details such as status, assignee etc., if id is empty then it returns all unassigned tasks and pending ones only.
# 3. `PATCH /api/tasks/{id}` - Update task like title or description; assigning new team members can be done in this end-point too by specifying which user to assign the changes to via JSON body payload of PATCH request, mark as complete operation will also work here if status is changed from pending -> marked_complete.
# 4. `DELETE /api/tasks/{id}` - Delete a task based on id provided through GET or POST requests (user confirmation required). 
# 5. `POST /api/assignments` and `GET /api/assignments?userId={...}&teamId={...}`: Create assignment of tasks to specific team members for their individual performance tracking, get all assignments assigned within the last month etc., which also includes mark_as_complete as a special operation.
# 6. `POST /api/reminders` - Send an automated reminder when deadline is approaching and not completed yet based on current time vs task due date comparison (automation logic handled at service level). 
# 7. The manager dashboard endpoint `/dashboard`, which will be secured via role-based access controls, to view the progress of tasks assigned with each team member's status within a specific project/department and also provide CSV report download functionality for task completion data (using Nodemailer integration if needed). 
# 8. The application is using OAuth2 protocol along with JSON Web Tokens (JWT) mechanism ensuring secure login access, which helps in role-based permissions control as well by checking roles embedded within JWT token payload at every protected endpoints request validation step to grant or deny operation permission based on user's authenticated identity and their assigned role.
# 9. Integration with existing email system for notifications is handled internally using Nodemailer (SMTP library) in our server-side Express backend, which sends out emails upon task creation/editing & assignment as well as when the reminders are triggered or reports requested by managers on dashboard based solethd of provided criteria.
# 10. Report generation functionality will be handled via `/api/reports` endpoint with CSV format output that can be downloaded directly from manager dashboard, which is built using React Native framework to ensure mobile compatibility as well due its lightweight nature and performance capabilities when it comes across multiple screen sizes (via responsive design principles). The reports themselves are generated on-the-fly based on the request parameters in query string or JSON body payload of POST requests.
# 11. For team member management, endpoints like `GET /api/team_members` & `POST /api/teams/{id}/member?userId={...}` would be included for managers to view all their assigned members and also add new ones respectively; this enables better performance tracking of individual user contributions as well.
# 12. The frontend UI design principles are based on responsive web practices, ensuring the application scales smoothly across desktop & mobile devices by using CSS Grid/Flexbox techniques in combination with React component libraries like Material-UI or Ant Design to maintain consistent look and feel of various elements throughout multiple screen sizes while keeping accessibility standards (WCAG) as well.
# 13. Finally, data validation and business logic are encapsulated at the service layer using middleware functions that filter all incoming requests before they reach API endpoints for security checks/permissions based on roles etc., to prevent any unauthorized or malicious actions by potential attackers trying their luck through exposed web services (which also helps in improving system performance due reduction of unnecessary computation at request validation step). 
# 14. Error handling mechanism is implemented via custom error handlers & exception classes built within the Node server code, which provides detailed and user-friendly feedback messages whenever something goes wrong with requests/responses during application usage or data processing; this helps in maintaining good user experience along as well while ensuring necessary audit trails for troubleshooting purposes.
# 15. The entire system architecture is designed based on principles of scalability, security & performance optimization so that it can handle heavy traffic loads with minimal latency/delay during peak hours or eventful scenarios (either in terms of user interactions within application itself OR through third-party integrations). This ensures reliability for both internal company usage as well when partnering organizations get involved via API calls; all the while keeping system architecture clean & maintainable by following best practices such as modularity, code splitting/lazy loading etc. 
# 16. Code documentation is maintained using JSDoc comments within JavaScript files to help future developers understand how specific components work together along with some inline examples wherever needed for quick reference; this also helps in reducing development turnaround times by allowing new team members or external contractors easier access without much context setup required initially due familiarity gained through well-documented codebase.
# 17. Continuous integration/continuous delivery (CI/CD) pipeline is established via Jenkins using GitHub source control repository; this provides automated testing & deployment scripts that run nightly builds along with running suite of unit tests to validate functionality changes, enhancements or fixes before pushing any new features into master branch ready for production release at scheduled intervals based on business needs.
# 18. Last but not least - monitoring tools like Prometheus alongside Grafana dashboards setup within the backend server environment will ensure proactive performance tracking & alerting capabilities when anomalies are detected (either in terms of resource utilization or user interaction patterns); this helps maintain system health continuously by providing real-time insights on various metrics along as well which also serves to identify potential areas for optimization/improvements down the line based off actual usage data captured during operation hours. This overall robust & effective architecture specification provides all necessary components required within our task management web application while ensuring secure access controls, efficient collaboration features among team members etc., meeting given objectives along as well optimizing user experience across multiple devices/screen sizes without compromising on any quality assurance aspect throughout entire development lifecycle.

# """

# prompt = code_instruction + arch_text

# def query_model(model, prompt):
#     """Send prompt to Ollama model and return response"""
#     try:
#         response = requests.post(OLLAMA_URL, json={
#             "model": model,
#             "prompt": prompt,
#             "stream": False
#         })
#         response.raise_for_status()
#         return response.json().get("response", "").strip()
#     except Exception as e:
#         return f"❌ Error with {model}: {str(e)}"

# def score_code_output(output):
#     """
#     Score the code output for:
#     1. Section presence (frontend, backend, database)
#     2. JSON validity
#     3. Word length (code length)
#     4. Accuracy: coverage of key architecture features
#     """
#     # 1️⃣ Section check
#     sections = ["frontend", "backend", "database"]
#     try:
#         output_json = json.loads(output)
#         section_score = sum(1 for sec in sections if sec in output_json and len(output_json[sec]) > 0)
#     except json.JSONDecodeError:
#         section_score = 0

#     # 2️⃣ JSON validity check
#     json_score = 1 if section_score > 0 else 0

#     # 3️⃣ Length check (total code lines across all files)
#     try:
#         total_lines = sum(len(str(output_json[sec][file]).splitlines()) for sec in output_json for file in output_json[sec]) if json_score else 0
#         length_score = 1 if 100 <= total_lines <= 500 else 0
#     except:
#         length_score = 0

#     # 4️⃣ Accuracy check (keywords from architecture)                  ----------------Issue in this coz, now we are updating the architecture doc maunally, so it cant meet the criteria if we manually gave the key features. so it should be dynamic..
#     key_features = [
#         "React", "Redux", "Express", "Sequelize", "PostgreSQL", "Nodemailer", "JWT", "OAuth2",
#         "POST /api/tasks", "GET /api/tasks", "PATCH /api/tasks", "DELETE /api/tasks",
#         "POST /api/assignments", "GET /api/assignments", "POST /api/reminders",
#         "GET /api/reports", "POST /api/auth/login", "/dashboard",
#         "users table", "tasks table", "team_members table", "roles table", "reports table",
#         "email notifications", "CSV report", "role-based access"
#     ]
#     matched_features = sum(1 for feat in key_features if feat.lower() in output.lower())
#     accuracy_score = matched_features / len(key_features)  # normalized 0–1

#     # 5️⃣ Total score (sections + json + length + accuracy)
#     total_score = section_score + json_score + length_score + accuracy_score * 4  # max=3+1+1+4=9
#     return total_score, section_score, total_lines, accuracy_score

# # Run models and store results
# results = []

# for model in models:
#     print("="*60)
#     print(f"🚀 Model: {model}")
#     print("-"*60)
#     output = query_model(model, prompt)
#     print(output)
#     total, sec_score, lines, acc_score = score_code_output(output)
#     print(f"✅ Total Score: {total:.2f}/9 | Sections: {sec_score}/3 | Lines: {lines} | Accuracy: {acc_score*100:.0f}%\n")
#     results.append((model, total))

# # Rank models
# results.sort(key=lambda x: x[1], reverse=True)
# best_model = results[0][0]

# print("="*60)
# print(f"🏆 Best model for Agent 3: {best_model}")